# Session 11: Advanced Retrieval with LangChain

## Learning Objectives:

- Understand and implement multiple retrieval strategies for RAG
- Compare naive, BM25, multi-query, parent-document, contextual compression, ensemble, and semantic chunking approaches
- Build RAG chains over a health and wellness knowledge base using LangChain and QDrant

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

---

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

> NOTE: Create a `.env` file in this directory with `OPENAI_API_KEY` and `COHERE_API_KEY` to avoid being prompted each time.

In [1]:
# import os
# import getpass
# from dotenv import load_dotenv

# load_dotenv()

# if not os.environ.get("OPENAI_API_KEY"):
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")
# Load environment variables from .env file
from uuid import uuid4
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

cohere_key = get_api_key("COHERE_API_KEY", "Cohere API Key: ")
if cohere_key:
    print("cohere API key set")
else:
    print("Warning: No Cohere API key configured")

# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

if not os.environ.get("METAL_API_KEY"):
    os.environ["METAL_API_KEY"] = getpass("Please enter your metals.dev API key!")

cohere API key set
Anthropic API key set
OpenAI API key set
LangSmith tracing enabled. Project: AIE9 - Deep Agents - 1a1bbd9b


In [2]:
# if not os.environ.get("COHERE_API_KEY"):
#     os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Health and Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, stress management, habits, and common health concerns.

### Data Preparation

We'll load the wellness guide as a single document, then split it into smaller chunks using a `RecursiveCharacterTextSplitter` for our vector store. We also keep the raw (unsplit) document for use with the Parent Document Retriever and Semantic Chunker later.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

Let's verify our data was loaded and split correctly!

In [4]:
print(f"Raw documents: {len(raw_docs)}")
print(f"Split chunks: {len(wellness_docs)}")
print(f"\nExample chunk:\n{wellness_docs[0]}")

Raw documents: 1
Split chunks: 45

Example chunk:
page_content='The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.' metadata={'source': 'data/HealthWellnessGuide.txt'}


## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "wellness_guide".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [5]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [10]:
naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on hands and knees, alternate arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while engaging your core. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\nThese exercises are gentle and specifically aimed at alleviating lower back discomfort. Always consult 

In [11]:
naive_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a crucial role in overall health by supporting physical repair, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that help regulate growth and appetite. Adequate sleep, typically 7-9 hours per night for adults, is essential for a strong immune system, emotional stability, and optimal brain function. Poor sleep or sleep disorders like insomnia can negatively impact health, increasing the risk of chronic conditions, impairing immune response, and affecting mood and cognitive abilities. Maintaining good sleep hygiene, such as following a consistent routine and creating a sleep-friendly environment, is important for promoting overall health and well-being.'

In [12]:
naive_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing, progressive muscle relaxation, grounding techniques, taking short walks in nature, listening to calming music, and using essential oils like peppermint or lavender. For headaches specifically, staying well-hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, and gentle massage of the temples and neck can also be effective.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [13]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(wellness_docs)

We'll construct the same chain - only changing the retriever.

In [14]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [15]:
bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, then alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent, then flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back pain and may prevent future episodes.'

In [16]:
bm25_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a crucial role in overall health by supporting various bodily functions. According to the information provided, maintaining a regular sleep schedule, creating a comfortable sleep environment, and practicing good sleep hygiene can enhance sleep quality. Quality sleep helps with physical health, mental well-being, immunity, and energy levels. Poor or inadequate sleep can negatively affect your health, potentially leading to issues like insomnia, decreased immunity, and impaired mental health. Therefore, prioritizing good sleep habits is essential for maintaining overall wellness.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include relaxation techniques such as progressive muscle relaxation, meditation, and deep breathing exercises. Herbal teas like chamomile or valerian root may help reduce stress. Ensuring adequate hydration and good sleep hygiene can also be beneficial in managing headaches related to stress.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### Answer:
BM25 is better exact text search so very particular things like ERROR codes, Product SKUs, or very specific phrasings will match.
If you want to find the SKU to replace that part to your Gas Grill (true story) then you need to know its MasterCraft #8dsfLLQR-big.  BM25 will find that exact part in your dataset exactly, vs another algorithm which may just return vague phrases about 'grill repair'.  IF you wanted to know the real name of it, or the website to get a replacement then getting to the exact SKU or part number is important.


## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [18]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
# naive retriever takes 20 docs
# reranker takes 20 docs and returns 5
# it will see if the question, when re-asked.. do these documents answer this question?
# stack rank the goodness of those returns
compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [19]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [20]:
contextual_compression_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'The exercises that can help with lower back pain include:\n\n1. **Cat-Cow Stretch:** Start on your hands and knees. Alternate between arching your back up (like a cat) and letting it sag down (like a cow). Do 10-15 repetitions.\n\n2. **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold the position for 5 seconds, then switch sides. Perform 10 repetitions per side.\n\n3. **Pelvic Tilts:** Lie on your back with knees bent. Flatten your back against the floor by tightening your abdominal muscles and tilting your pelvis slightly upward. Hold for 10 seconds and repeat 8-12 times.\n\nAlways consult with a healthcare professional before starting any new exercise routine, especially if you have existing health conditions.'

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly affects overall health by supporting physical repair, mental well-being, and cognitive function. During sleep, the body repairs tissues and releases hormones that regulate growth and appetite. Adequate sleep, typically 7-9 hours per night for adults, also consolidates memories and promotes mental clarity. Poor sleep or disorders like insomnia can negatively impact health, leading to issues with mood, immunity, and overall functioning. Creating a comfortable sleep environment and maintaining good sleep habits are important for maximizing these health benefits.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include deep breathing exercises, progressive muscle relaxation, grounding techniques, taking short walks in nature, listening to calming music, and applying essential oils like peppermint or lavender. For headaches specifically, staying hydrated, resting in a dark, quiet room, gentle massage of the temples and neck, and using cold or warm compresses can also help.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [23]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [24]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help alleviate lower back pain include:\n\n- Cat-Cow Stretch: Start on your hands and knees, alternate between arching your back up (cat position) and letting it sag down (cow position). Perform 10-15 repetitions.\n\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abdominal muscles and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle exercises can help improve flexibility and strengthen the muscles supporting your lower back.'

In [26]:
multi_query_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is crucial for physical health, mental well-being, and cognitive function. During sleep, your body repairs tissues, consolidates memories, and regulates hormones that control growth and appetite. Adequate sleep—typically 7 to 9 hours for adults—supports a strong immune system, helps manage stress, and improves mood and mental clarity. Poor sleep or sleep disorders like insomnia can lead to health issues such as fatigue, headaches, increased stress, and a decline in physical and mental health. Therefore, maintaining good sleep hygiene and creating an optimal sleep environment are essential for overall health and well-being.'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Deep breathing exercises (e.g., inhale for 4 counts, hold for 4, exhale for 4)\n- Progressive muscle relaxation (tensing and releasing muscle groups)\n- Grounding techniques (naming things you see, hear, feel, smell, taste)\n- Taking a short walk, preferably in nature\n- Listening to calming music\n- Drinking water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using essential oils like peppermint or lavender\n\nAdditionally, maintaining a regular sleep schedule, practicing mindfulness or meditation, and engaging in hobbies or relaxation activities can help manage stress and headaches naturally.'

### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### Answer:

Recall is making sure you answer everything, even if sometimes its not perfect.
This means that you're optimizing for having an answer.
Formulating a question in many ways improves recall because the phrases 'make it bigger' and 'get a larger sample' kinda mean a similar thing.  By having more coverage of a concept in the vector space you're more likely to have a match when the user for example types 'make it bigger' you're going to have a direct match in the data set.

So when you generate a response in multiple different phrasings, you're likely to find a match that may not have been retrieved by the users query.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. We split the full document into large "parent" chunks (e.g. 2000 characters).
2. Each parent chunk is further split into smaller "child" chunks (e.g. 400 characters).
3. The child chunks are stored in a VectorStore, while the parent chunks are stored in an in-memory docstore.
4. When we query our Retriever, we do a similarity search comparing our query vector to the child chunks.
5. Instead of returning the child chunks, we return their associated parent chunks.

The basic idea is:

- **Search** for small, focused chunks (better semantic matching)
- **Return** big chunks (richer surrounding context)

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by defining our parent and child splitters.

In [28]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [29]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="wellness_parent_child",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="wellness_parent_child", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [30]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [31]:
parent_document_retriever.add_documents(raw_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [32]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [33]:
parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'To help with lower back pain, the recommended exercises include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Complete 10 repetitions per side.\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over your chest, tighten your stomach muscles, and raise your shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises

In [34]:
parent_document_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical health, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that help regulate growth and appetite. Adults generally need 7-9 hours of quality sleep per night. Good sleep supports a healthy immune system, boosts energy levels, improves mood, and enhances learning and memory. Poor sleep or sleep deprivation can lead to fatigue, weakened immunity, mood disturbances, and increased risk of chronic conditions such as cardiovascular disease and diabetes. Therefore, maintaining healthy sleep habits is crucial for overall wellness.'

In [35]:
parent_document_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing techniques, engaging in relaxation exercises like progressive muscle relaxation, and mindfulness or meditation to promote mental calmness. For headaches specifically, remedies such as applying a cold or warm compress to the head or neck, resting in a dark, quiet room, gentle massage of the temples and neck, and using essential oils like peppermint or lavender can be helpful. Staying well-hydrated by drinking water is also important, as dehydration can trigger headaches. Additionally, maintaining a regular sleep schedule and managing stress through hobbies, social connections, and setting boundaries can contribute to reducing stress and associated headaches.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [36]:
from langchain.retrievers import EnsembleRetriever
# not reranking
# taking many ranked lists and combining them into one ranked list
retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [37]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [38]:
ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n1. Cat-Cow Stretch: Start on your hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n2. Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for 5 seconds, then switch sides. Do 10 repetitions per side.\n3. Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds. Repeat 8-12 times.\n4. Partial Crunches: Lie on your back with knees bent, cross arms over your chest, tighten your stomach muscles, and raise your shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n5. Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\nThese exercises are gentle and aimed at relieving lower back discomf

In [39]:
ensemble_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical, mental, and cognitive functions. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep, generally 7-9 hours per night, helps strengthen the immune system, improve mood, and enhance concentration and learning. Poor or insufficient sleep can lead to issues such as increased stress, weakened immunity, mood disturbances, and impaired cognitive performance. Maintaining good sleep hygiene and creating an optimal sleep environment are important strategies for ensuring restorative sleep and promoting overall wellness.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Deep breathing exercises (e.g., inhaling for 4 counts, holding, exhaling, and repeating)\n- Progressive muscle relaxation by tensing and releasing muscle groups\n- Grounding techniques, such as naming things you see, hear, feel, smell, and taste\n- Taking short walks, preferably in nature\n- Listening to calming music\n- Using essential oils like peppermint or lavender\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of temples and neck\n\nThese methods can help alleviate stress and reduce headache symptoms naturally.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [41]:
from langchain_experimental.text_splitter import SemanticChunker
# sentences are related to one another.. 
# paragraphs are probaly coherent
# semantic chunking takes every sentence and each expresses one idea or thought
# so we break the document into chunks that each express one idea or thought, if 2 sentences express the same idea, they are combined into one chunk
semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [42]:
semantic_documents = semantic_chunker.split_documents(raw_docs)

Let's create a new vector store.

In [43]:
semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide_semantic_chunks"
)

We'll use naive retrieval for this example.

In [44]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [45]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [46]:
semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help alleviate lower back pain include:\n\n- **Cat-Cow Stretch:** Begin on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n\n- **Partial Crunches:** Lie on your back with knees bent, arms crossed over chest, tighten your stomach muscles, and lift shoulders off the floor. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, and repeat 8-12 times.\n\nThese exercises are recommended for gentle stretching and strengthening to help relieve lower back discomfort and prevent future issues.'

In [47]:
semantic_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical restoration, mental well-being, and cognitive functions. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep, typically 7-9 hours for adults, is essential for maintaining energy levels, immune function, hormone balance, and emotional health. Poor sleep quality or insufficient sleep can lead to issues such as fatigue, headaches, weakened immunity, and mental health problems like stress and anxiety. Therefore, prioritizing good sleep hygiene and creating a conducive sleep environment are crucial for overall health and wellness.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- For stress:\n  - Deep breathing exercises (e.g., inhaling for 4 counts, holding for 4, exhaling for 4)\n  - Progressive muscle relaxation (tensing and releasing muscle groups)\n  - Grounding techniques (naming things you see, hear, feel, smell, and taste)\n  - Taking short walks, especially in nature\n  - Listening to calming music\n  - Regular practice of mindfulness and meditation\n\n- For headaches:\n  - Staying well-hydrated by drinking water\n  - Applying cold or warm compresses to the head or neck\n  - Resting in a dark, quiet environment\n  - Gentle massage of temples and neck muscles\n  - Using essential oils like peppermint or lavender\n  - Managing triggers such as dehydration, stress, poor sleep, skipped meals, and eye strain\n\nThese natural approaches can help alleviate both stress and headaches effectively.'

### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### Answer:
Since Semantic chunking takes realted ideas and co-locates them in the resulting vector store then the small and highly repetetive ideas will all be near one another.  Their similarity will be very low and the phrases will be interchangeable, which can cause issues like:
* with many phrases that are almost the size, you may get them into one large chunk with no similarity or if they are broken apart into different chunks they may be abitrarily broken since theres no clear signal on how to break them.
* if the phases are very short they don't have much signal, so the embedding carries less distinctive information.  The difference between real meaning and random noise is hart to detect.

There are a few ways to adjust:
* If the content is fixed size then you can try to break every Q&A into one chunk by itself, and avoid random splits and mixing chunks.
* Lower the similarity threshold (or use a percentile-based breakpoint). Instead of a fixed cosine similarity cutoff, split at relative drops — e.g., wherever similarity falls below the 25th percentile of all adjacent-pair scores in the document. This adapts to the narrower range of similarity values you see in repetitive content.
* Group by structural cues. Something like an FAQs will have clear question/answer pairs. If we Pre-process with regexp markers (question marks, heading tags, numbered items) before applying any semantic logic then semantic chunking becomes a second pass to merge pairs rather than the primary segmentation strategy.
* Increase the window size. Instead of comparing single sentences, compare rolling windows of 2–3 sentences. This gives the embeddings more context and produces more differentiated vectors, making real topic boundaries easier to detect.




---

# 🤝 Breakout Room Part #2

### 🏗️ Activity #1:

Your task is to evaluate the various Retriever methods against each other.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparison between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [49]:
# Simplified method from Unit 10

from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

dataset.to_pandas()


/var/folders/nw/60523x1n0jdfjd_q0kccg30w0000gn/T/ipykernel_9861/1805484729.py:12: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
/var/folders/nw/60523x1n0jdfjd_q0kccg30w0000gn/T/ipykernel_9861/1805484729.py:13: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())


Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]

...

Skipping multi_hop_abstract_query_synthesizer due to unexpected error: No relationships match the provided condition. Cannot form clusters.


Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,How do Chin Tucks help neck?,[PART 1: EXERCISE AND MOVEMENT\n\nChapter 1: U...,Chin Tucks help relieve neck and shoulder tens...,Wellness Enthusiast,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
1,How do Bird Dog help my back?,[PART 1: EXERCISE AND MOVEMENT\n\nChapter 1: U...,"Bird Dog is done from hands and knees, you ext...",Wellness Enthusiast,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
2,whts in chpter 5 and how do i use it for meal ...,[PART 2: NUTRITION AND DIET\n\nChapter 4: Fund...,Chapter 5: Meal Planning for Wellness explains...,Wellness Enthusiast,MISSPELLED,LONG,single_hop_specific_query_synthesizer
3,What key hydration tips are provided in Chapte...,[PART 2: NUTRITION AND DIET\n\nChapter 4: Fund...,Chapter 6 suggests several tips for staying hy...,Wellness Enthusiast,PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
4,Wut is PART 5 about and how can it help me bil...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,PART 5 focuses on building healthy habits by e...,Wellness Enthusiast,MISSPELLED,MEDIUM,single_hop_specific_query_synthesizer
5,whts in chpter 18 for boosting imune functon a...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 18 focuses on boosting immune function...,Wellness Enthusiast,MISSPELLED,LONG,single_hop_specific_query_synthesizer
6,Wht are the main sleep hygeene practises discu...,[<1-hop>\n\nPART 2: NUTRITION AND DIET\n\nChap...,Chapter 8 discusses essential sleep hygiene pr...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
7,How come Chapter 7 say sleep is important for ...,[<1-hop>\n\nPART 2: NUTRITION AND DIET\n\nChap...,Chapter 7 say sleep is crucial for physical he...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
8,Wht foods in Chaptar 17 help gut helth and how...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,"In Chapter 17, foods that help gut health incl...",NaN,NaN,NaN,multi_hop_specific_query_synthesizer
9,What practical sleep hygiene practices and env...,[<1-hop>\n\nPART 2: NUTRITION AND DIET\n\nChap...,Chapter 8 recommends several practical sleep h...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer


In [50]:
# # By hand method from Unit 9

# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

# from langchain_community.document_loaders import DirectoryLoader, TextLoader

# loader = DirectoryLoader("data/", glob="*.txt", loader_cls=TextLoader)
# docs = loader.load()
# print(f"Loaded {len(docs)} documents: {[d.metadata['source'] for d in docs]}")

# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper
# from langchain_openai import ChatOpenAI
# from langchain_openai import OpenAIEmbeddings
# generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
# generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

# from ragas.testset.graph import KnowledgeGraph

# kg = KnowledgeGraph()
# kg
# from ragas.testset.graph import Node, NodeType

# for doc in docs:
#     kg.nodes.append(
#         Node(
#             type=NodeType.DOCUMENT,
#             properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
#         )
#     )
# kg

# from ragas.testset.transforms import default_transforms, apply_transforms

# transformer_llm = generator_llm
# embedding_model = generator_embeddings

# default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
# apply_transforms(kg, default_transforms)
# kg

# kg.save("usecase_data_kg.json")
# usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
# usecase_data_kg

# from ragas.testset import TestsetGenerator

# generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

# from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

# query_distribution = [
#         (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
#         (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
#         (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
# ]

# testset = generator.generate(testset_size=10, query_distribution=query_distribution)
# testset.to_pandas()



## HW: Results
The below cell implements a loop where we walk through the multiple retriever chains against a golden dataset we produced above.  It stores them all in a LangSmith experiment

Heres a quick table to analyze the results:

| Retriever | Cost | Latency | 
|---|---|---|
| Naive |.61 |432 sec | 
| BM25 |.30 |224 sec | 
| Contextual Compression |.29 | 212 sec |
| Parent Document  |.27 |151 sec |
| Multi-Query|.70 |455 sec | 
| Semantic |.54 |370 sec | 
| Ensamble |.63 |449 sec | 

### Performance Summary (averaged across 12 test samples)

| Metric | Naive | BM25 | Ctx Compression | Parent Doc | Multi-Query | Ensemble | Semantic |
|---|---|---|---|---|---|---|---|
| Answer Relevancy | 0.944 | 0.701 | 0.941 | 0.940 | 0.943 | 0.944 | **0.949** |
| Context Recall | **1.000** | 0.410 | 0.851 | 0.979 | **1.000** | **1.000** | 0.910 |
| Context Entity Recall | 0.414 | 0.161 | 0.304 | **0.436** | 0.293 | 0.343 | 0.186 |
| Factual Correctness | 0.659 | 0.413 | 0.614 | 0.672 | 0.682 | 0.684 | **0.689** |
| Faithfulness | 0.649 | 0.498 | 0.655 | 0.720 | 0.706 | 0.725 | **0.776** |
| Noise Sensitivity (lower=better) | **0.026** | 0.110 | 0.081 | 0.164 | 0.052 | 0.133 | 0.178 |

Note: Noise sensitivity had many null values for Naive (3/12), Multi-Query (7/12), and Ensemble (9/12), making those averages less reliable. **Bold** = best in category.

### Rankings (1st = best)

| Rank | Answer Relevancy | Context Recall | Entity Recall | Factual Correct. | Faithfulness | Noise Sens. |
|---|---|---|---|---|---|---|
| 1st | Semantic | Naive/Multi/Ensemble (tie) | Parent | Semantic | Semantic | Naive* |
| 2nd | Naive | Parent | Naive | Ensemble | Ensemble | Multi* |
| 3rd | Ensemble | Semantic | Ensemble | Multi | Parent | Ctx Compress |
| 4th | Multi | Ctx Compress | Ctx Compress | Parent | Multi | BM25 |
| 5th | Ctx Compress | BM25 | Multi | Naive | Ctx Compress | Ensemble* |
| 6th | Parent | - | Semantic | Ctx Compress | Naive | Parent |
| 7th | BM25 | - | BM25 | BM25 | BM25 | Semantic |

### Key Takeaways

- **Semantic chunking** leads on 3 of 6 metrics (answer relevancy, factual correctness, faithfulness) - strongest overall on generation quality. Included for completeness - realizing this is not itself a retrieval strategy.
- **Ensemble** is a strong contender - perfect context recall (1.0), 2nd in faithfulness (0.725) and factual correctness (0.684). Combines the strengths of BM25 keyword matching with dense vector retrieval.
- **Naive, Multi-Query, and Ensemble** all achieve perfect context recall (1.0) - but multi-query costs the most ($0.70) and is slowest (455s).
- **Parent Document** is the best balance of cost ($0.27), speed (151s), and strong performance across all metrics - a solid all-rounder.
- **BM25** is fast and cheap but significantly underperforms on context recall (0.41) and answer relevancy (0.70) - keyword matching alone misses semantically relevant content.
- **Contextual Compression (Rerank)** lands in the middle of the pack on most metrics despite being cost-efficient ($0.29, 212s).



In [ ]:
# ok now given all the retriever chains
# make each an experiment
# cut them loose on our testset
# collect the results
# run them through ragas
# use LangSmith to collect the results

# bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content
# naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content
# parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content
# ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content
# multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content
# semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

chains = {
    "naive": naive_retrieval_chain,
    "bm25": bm25_retrieval_chain,
    "contextual_compression": contextual_compression_retrieval_chain,
    "parent": parent_document_retrieval_chain,
    "multi": multi_query_retrieval_chain,
    "semantic": semantic_retrieval_chain,
    "ensemble": ensemble_retrieval_chain
}

from copy import deepcopy
import time

for chain_name, chain in chains.items():
  iteration_data = deepcopy(dataset)
  # Now Evaluate them!!
  samples = []
  for test_row in iteration_data:
    #response = graph.invoke({"question" : test_row.eval_sample.user_input})
    response = chain.invoke({"question" : test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"].content
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
    samples.append(test_row.eval_sample)
    time.sleep(5)

  from ragas import EvaluationDataset

  #evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
  evaluation_dataset = EvaluationDataset(samples=samples)

  from ragas import evaluate
  from ragas.llms import LangchainLLMWrapper

  evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

  from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
  from ragas import evaluate, RunConfig

  custom_run_config = RunConfig(timeout=360)

  baseline_result = evaluate(
      dataset=evaluation_dataset,
      metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
      llm=evaluator_llm,
      run_config=custom_run_config,
      experiment_name=f"WEEK_11_CHAIN_COMPARISON_{chain_name}"
  )
  baseline_result



/var/folders/nw/60523x1n0jdfjd_q0kccg30w0000gn/T/ipykernel_9861/243628073.py:48: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/var/folders/nw/60523x1n0jdfjd_q0kccg30w0000gn/T/ipykernel_9861/243628073.py:50: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
/var/folders/nw/60523x1n0jdfjd_q0kccg30w0000gn/T/ipykernel_9861/243628073.py:50: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated a

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[29]: TimeoutError()
Exception raised in Job[35]: TimeoutError()
Exception raised in Job[41]: TimeoutError()
Exception raised in Job[47]: TimeoutError()
Exception raised in Job[53]: TimeoutError()
Exception raised in Job[59]: TimeoutError()
Exception raised in Job[65]: TimeoutError()
Exception raised in Job[71]: TimeoutError()
